<a href="https://www.kaggle.com/code/anandkumar840972/emotion-recognisation-transfer-learning-technique?scriptVersionId=132647075" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> 

# 1.Import all the libraries thats needed

In [2]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
Image_size=[224,224]

In [4]:
train_path="/kaggle/input/emotion-detection-fer/train"

In [5]:
test_path="/kaggle/input/emotion-detection-fer/test"

In [7]:
vgg=VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=Image_size+[3],
       )

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
for layer in vgg.layers:
    layer.trainable=False

In [ ]:
training_dataset

# number of the classes we will find..


In [14]:
folders=glob("/kaggle/input/emotion-detection-fer/train/*")

In [16]:
#Flatten the output off vgg

x=Flatten()(vgg.output)

In [15]:
print(len(folders))

7


In [18]:
predictions=Dense(len(folders), activation='softmax')(x)

In [24]:
#create the object of the model that are used
model=Model(inputs=vgg.input,outputs=predictions)

In [25]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [21]:
from tensorflow.keras.layers import Input, Dense

In [27]:
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

In [31]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range= 0.2,
                                horizontal_flip=True)

training_set=train_datagen.flow_from_directory('/kaggle/input/emotion-detection-fer/train',
                                              target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical'
                                              )

Found 28709 images belonging to 7 classes.


In [32]:
test_datagen= ImageDataGenerator(rescale=1./255)

testing_set=test_datagen.flow_from_directory('/kaggle/input/emotion-detection-fer/test',
                                              target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical'
                                              )

Found 7178 images belonging to 7 classes.


Fitting the data


In [34]:
r=model.fit_generator(training_set,
                     validation_data=testing_set,
                     epochs=5,
                     steps_per_epoch=len(training_set),
                     validation_steps=len(testing_set))

/tmp/ipykernel_28/187259063.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r=model.fit_generator(training_set,


Epoch 1/5
898/898 [==============================] - 656s 711ms/step - loss: 1.5795 - accuracy: 0.4215 - val_loss: 1.6102 - val_accuracy: 0.4464
Epoch 2/5
898/898 [==============================] - 410s 456ms/step - loss: 1.4493 - accuracy: 0.4815 - val_loss: 1.3831 - val_accuracy: 0.4974
Epoch 3/5
898/898 [==============================] - 400s 446ms/step - loss: 1.3678 - accuracy: 0.5046 - val_loss: 1.4119 - val_accuracy: 0.5011
Epoch 4/5
898/898 [==============================] - 415s 462ms/step - loss: 1.3440 - accuracy: 0.5209 - val_loss: 1.8753 - val_accuracy: 0.4262
Epoch 5/5
898/898 [==============================] - 413s 460ms/step - loss: 1.3078 - accuracy: 0.5319 - val_loss: 1.4837 - val_accuracy: 0.4894


In [44]:
import tensorflow as tf
from keras.models import load_model
model.save("Emotions_model.h5")